# JupyterHub Notebook

### This notebook server is hosted on the OpenShift platform which provides a separate server for each individual user. The platform takes care of the provisioning of the server and allocating related to storage.

### First, install and import required libraries, watermark our file, initialise our Spark Session Builder and initialise our environment with required configuration

In [2]:
%pip install watermark
%pip install Minio
%pip install matplotlib


You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 11.3 MB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 880 kB 134.8 MB/s eta 0:00:01
     |████████████████�^C███████▏     | 12.9 MB 125.3 MB/s eta 0:00:01

ERROR: Operation cancelled by user
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import json
from pyspark import SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import from_json, col, to_json, struct
import watermark
from minio import Minio

%matplotlib inline
%load_ext watermark

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
%watermark -n -v -m -g -iv


In [ ]:
sparkSessionBuilder = SparkSession\
    .builder\
    .appName("Customer Churn ingest Pipeline")

In [6]:
submit_args = "--conf spark.jars.ivy=/tmp \
--conf spark.hadoop.fs.s3a.endpoint=http://minio-ml-workshop:9000 \
--conf spark.hadoop.fs.s3a.access.key=minio \
--conf spark.hadoop.fs.s3a.secret.key=minio123 \
--conf spark.hadoop.fs.s3a.path.style.access=true \
--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem \
--packages org.apache.hadoop:hadoop-aws:3.2.0"



###  Connect to Spark Cluster provided by OpenShift Platform

In [ ]:
import spark_util

spark = spark_util.getOrCreateSparkSession("ML Ops Demo", submit_args)
spark.sparkContext.setLogLevel("INFO")
print('Spark context started.')

###  Declare our input data sources, import and combine them

In [ ]:
dataFrame_Customer = spark.read\
                .options(delimeter=',', inferSchema='True', header='True') \
                .csv("s3a://rawdata/customers/Customer-Churn_P1.csv")
dataFrame_Customer.printSchema()

In [ ]:
dataFrame_Products = spark.read\
                .options(delimeter=',', inferSchema='True', header='True') \
                .csv("s3a://rawdata/products/Customer-Churn_P2.csv")
dataFrame_Products.printSchema()


In [ ]:
# from pyspark.sql.types import *
# from  pyspark.sql.functions import *

# srcKafkaBrokers = "odh-message-bus-kafka-bootstrap:9092"
# srcKakaTopic = "datatelco"



# schema = StructType()\
#     .add("customerID", IntegerType())\
#     .add("PhoneService", StringType())\
#     .add("MultipleLines", StringType())\
#     .add("InternetService", StringType())\
#     .add("OnlineSecurity", StringType())\
#     .add("OnlineBackup", StringType())\
#     .add("DeviceProtection", StringType())\
#     .add("TechSupport", StringType())\
#     .add("StreamingTV", StringType())\
#     .add("StreamingMovies", StringType())\
#     .add("Contract", StringType())\
#     .add("PaperlessBilling", StringType())\
#     .add("PaymentMethod", StringType())\
#     .add("MonthlyCharges", StringType())\
#     .add("TotalCharges", DoubleType())\
#     .add("Churn", StringType())



# #Read from JSON Kafka messages into a dataframe
# dfKafka = spark.read.format("kafka")\
#     .option("kafka.bootstrap.servers", srcKafkaBrokers)\
#     .option("subscribe", srcKakaTopic)\
#     .option("startingOffsets", "earliest")\
#     .load()\
#     .withColumn("value", regexp_replace(col("value").cast("string"), "\\\\", "")) \
#     .withColumn("value", regexp_replace(col("value"), "^\"|\"$", "")) \
#     .selectExpr("CAST(value AS STRING) as jsonValue")\
#     .rdd.map(lambda row: row["jsonValue"])

# dfObj = spark.read.schema(schema).json(dfKafka)
# dfObj.printSchema()
# dfObj.show(n=2)


In [ ]:
dataFrom_All = dataFrame_Customer.join(dataFrame_Products, "customerID", how="full")

###  Push prepared data to object storage and stop Spark cluster to save resources
###  Note - be sure to change this user_id on the next line to your username (something in the range user1 ... user30)

In [ ]:
user_id = "user29"
file_location = "s3a://data/full_data_csv" + user_id
dataFrom_All.repartition(1).write.mode("overwrite")\
    .option("header", "true")\
    .format("csv").save(file_location)

In [ ]:
spark.stop()